# Thomas Fire Analysis

Author: Michelle Yiv

[Link to GitHub Repository](https://github.com/mmyiv/eds220-hwk4)

## About this notebook

#### Dataset description: 
California fire perimeter was published by CAL FIRE and contains information about fires in California. In addition to the geometries (perimeters) of each fire, there is information about the name, date, and acres burned.

Landsat data contains both x,y coordinates and variables containing colors (R, G, B, NIR, SWIR) for plotting. This dataset contains the Santa Barbara location and surrounding areas. 

#### Purpose:
The purpose of this notebook is to use Landsat data to visualize the Santa Barbara area and combine it with the Thomas Fire data to map the boundary of the fire. In addition to this, true and false color imagery is being used to visualize the scars from the fire.
    
#### Analysis highlight list:
Landsat data was read in using an extension python package, `rioxarray`. After basic data exploration, a set of maps were produced by selecting specific colors, converting them to an array and plotting. Both true and false color images were plotted to display the extent of scarring from the 2017 Thomas fire, and to emphasize the difference between true and false color.
    
#### References to datasets:
California Fire Perimeter.
Date of access: 11-19-2024.
[Link to data](https://catalog.data.gov/dataset/california-fire-perimeters-all-b3436)
Citation: “Data.gov,” Data.gov, 2024. https://catalog.data.gov/dataset/california-fire-perimeters-all-b3436 (accessed Nov. 19, 2024).

Landsat Data.
Date of access: 11-19-2024.
[Link to data](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)
Citation: “Microsoft Planetary Computer,” Microsoft.com, 2024. https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2 (accessed Nov. 19, 2024).

## Retreiving the boundary of the 2017 Thomas Fire

In [2]:
# Import libraries
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rioxr
import matplotlib.pyplot as plt

In [3]:
# Load in data
fp = os.path.join('data', 'California_Fire_Perimeters_(all).shp')
fire = gpd.read_file(fp)

In [ ]:
# Select 2017 Thomas Fire boundary by name and year
thomas_fire = fire[(fire['FIRE_NAME']=='THOMAS') & (fire['YEAR_']==2017)]

# Verify changes in GeoDataFrame
thomas_fire

# Verify changes visually
thomas_fire.plot()

## True and False Color Imagery of the Thomas Fire